# Markov Chains

## Pooh Shapes by Disney
### Problem: There are 4 Shape cards: Circle, Square, Triangle, Rectangle. The goal of the game is to get all four shape cards. A dice has six sides: circle, square, triangle, rectangle, any, remove. If you roll 'Any' you get any of the shapes you do not have. If you roll remove, remove one of the shapes you have. If you roll a shape you already have, nothing happens.

### The game is over when you have all 4 shape cards.

### FIND THE EXPECTED NUMBER OF ROLLS FOR A PLAYER TO COLLECT ALL SHAPES STARTING FROM 0.

Game is essentially a Markov Chain with probability matrix:

|        A      |        B      |        C      |        D      |        E      |
|       ---     |       ---     |       ---     |       ---     |       ---     |
| $\frac{1}{6}$ | $\frac{5}{6}$ |        0      |        0      |        0      |   
| $\frac{1}{6}$ | $\frac{1}{6}$ | $\frac{4}{6}$ |        0      |        0      | 
|        0      | $\frac{1}{6}$ | $\frac{2}{6}$ | $\frac{3}{6}$ |        0      | 
|        0      |        0      | $\frac{1}{6}$ | $\frac{3}{6}$ | $\frac{2}{6}$ | 
|        0      |        0      |        0      |        0      |        1      | 

with states A, B, C, D, E.

A = 0 shape, B = 1 shapes, C = 2 shapes, D = 3 shapes, E = 4 shapes

E is an absorbing state and also represents that the game is over because all 4 shapes are found.

We can solve using Markov Chain techniques...

...or we can also solve by simulation!

### Empirical Expected Number of Rolls

We will first determine the empirical expected number of rolls by simulaion and then compare with the theoretical expected number. To do this, we will use accumulation, functions, and numpy in Python. We will be simulating the game 100,000 times and find the average number of rolls.

In [36]:
#Run this code

import numpy

def ab(rolls): #function for how many rolls it takes to get from state A to state B
    x = numpy.random.rand() #generates random number from 0 to 1
    while x < 1/6: #1/6 probability of rolling "Remove" so we would still have 0 shape cards
        rolls = rolls + 1 #adds a roll to the count
        x = numpy.random.rand() #resets the random number
    else: #if we get the other 5/6 probability, we move onto state B
        rolls = rolls + 1 
        x = numpy.random.rand()
        return rolls
        
    
def bc(rolls): #function for how many rolls it takes to get from state B to state C
    x = numpy.random.rand()
    while x < 2/6: #2/6 probability of rolling "Remove" or a shape card we already have 
        if x < 1/6: #1/6 probability of rolling shape card we already have
            rolls = rolls + 1
            x = numpy.random.rand()
        else: #1/6 probability of rolling "Remove," moving us back to state A
            rolls = rolls + 1 
            rolls = ab(rolls)
            x = numpy.random.rand()
    else: #if we get the other 4/6 probability, we move onto state C
        rolls = rolls + 1
        x = numpy.random.rand()
        return rolls
    
def cd(rolls): #function for how many rolls it takes to get from state C to state D
    x = numpy.random.rand()
    while x < 3/6: #3/6 probability of rolling "Remove" or a shape card we already have 
        if x < 2/6: #2/6 probability of rolling shape card we already have
            rolls = rolls + 1
            x = numpy.random.rand()
        else: #1/6 probability of rolling "Remove," moving us back to state B
            rolls = rolls + 1
            rolls = bc(rolls)
            x = numpy.random.rand()
    else: #if we get the other 3/6 probability, we move onto state D
        rolls = rolls + 1
        x = numpy.random.rand()
        return rolls
    
def de(rolls): #function for how many rolls it takes to get from state D to state E
    x = numpy.random.rand()
    while x < 4/6: #4/6 probability of rolling "Remove" or a shape card we already have 
        if x < 3/6: #3/6 probability of rolling shape card we already have
            rolls = rolls + 1
            x = numpy.random.rand()
        else: #1/6 probability of rolling "Remove," moving us back to state C
            rolls = rolls + 1
            rolls = cd(rolls)
            x = numpy.random.rand()
    else: #if we get the other 2/6 probability, we move onto state E and finish the game
        rolls = rolls + 1
        x = numpy.random.rand()
        return rolls
    
total = 0
for i in range(1,100000): #simiulating 100000 times
    rolls = 0
    a = ab(rolls)
    b = bc(a)
    c = cd(b)
    d = de(c)
    total = total + d
    
answer = (total/100000)
print ("Through simulation, the expected number of rolls for a player to collect all shapes starting from 0 shapes is", answer)

Through simulation, the expected number of rolls for a player to collect all shapes starting from 0 shapes is 9.8958


### Theoretical Expected Number of Rolls

Next, we will be solving the theoretical expected number of rolls using Markov Chains techniques.

$\mu_{xy}$ represents the number of steps/rolls it takes to go from state x to state y.

Therefore, we must solve this system of equations:

$\mu_{04}$ = 1 + $\frac{1}{6}$$\mu_{04}$ + $\frac{5}{6}$$\mu_{14}$

$\mu_{14}$ = 1 + $\frac{1}{6}$$\mu_{04}$ + $\frac{1}{6}$$\mu_{14}$ + $\frac{4}{6}$$\mu_{24}$

$\mu_{24}$ = 1 + $\frac{1}{6}$$\mu_{14}$ + $\frac{2}{6}$$\mu_{24}$ + $\frac{3}{6}$$\mu_{34}$

$\mu_{34}$ = 1 + $\frac{1}{6}$$\mu_{24}$ + $\frac{3}{6}$$\mu_{34}$

*$\mu_{44}$ = 0 because you are already at state 4 so it takes 0 steps.*


Rearranging these equations we get the following proper format to solve equations:

$\frac{5}{6}$$\mu_{04}$ - $\frac{5}{6}$$\mu_{14}$ = 1 

-$\frac{1}{6}$$\mu_{04}$ + $\frac{5}{6}$$\mu_{14}$ - $\frac{4}{6}$$\mu_{24}$ = 1 

-$\frac{1}{6}$$\mu_{14}$ + $\frac{4}{6}$$\mu_{24}$ - $\frac{3}{6}$$\mu_{34}$ = 1 

-$\frac{1}{6}$$\mu_{24}$ + $\frac{3}{6}$$\mu_{34}$ = 1 

In [35]:
#run this code
equations = numpy.array([[5/6, -5/6, 0, 0], [-1/6, 5/6, -4/6, 0], [0, -1/6, 4/6, -3/6], [0, 0, -1/6, 3/6]])    
solver = numpy.array([1,1,1,1])
solved = numpy.linalg.solve(equations, solver)
solved

array([ 9.9,  8.7,  6.9,  4.3])

In our array, 'solved,' the terms represent $\mu_{04}$, $\mu_{14}$, $\mu_{24}$, $\mu_{34}$

**Therefore, through theoretical calculation using Markov Chain techniques, we find that the expected rolls for a player to collect all shapes starting from 0 is $\mu_{04}$ or 9.9. This is essentially what we got when we ran our simulation.**